In [1]:
import visualizacion
import numpy as np
import torch
from torch import optim
import time
from torch.utils import data
from torch import nn

Z_LATENT_SPACE = 128
Z_INTERN_SPACE = 136 #256

#G_LR = 0.0002
#D_LR = 0.001
G_LR = 0.00002 
D_LR = 0.000002
EPOCHS = 5000
BSIZE = 32
CUBE_LEN = 64
BETAS = (0.9, 0.999)


#losses
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from FragmentDataset import FragmentDataset
import network_vox as nv
from torchvision import transforms as tfs

In [2]:
from pyvox.models import Vox
from pyvox.writer import VoxWriter

In [3]:
available_device = 'cpu'#"cuda" if torch.cuda.is_available() else "cpu"

In [4]:
G_encode_decode = nv._G_encode_decode(z_latent_space=Z_LATENT_SPACE, z_intern_space=Z_INTERN_SPACE).to(available_device)
D_encode_decode = nv._D().to(available_device)

In [5]:
dt = FragmentDataset('./data', 'test', )


In [6]:
data_loader = data.DataLoader(dt, batch_size=BSIZE, shuffle=False, drop_last=True)

In [7]:
PATH = 'weight/v2_G_encode_decode_partial_20.pkl'
checkpoint = torch.load(PATH)
G_encode_decode.load_state_dict(checkpoint)

<All keys matched successfully>

In [8]:
for i,  (mesh_frag, mesh_complete) in enumerate(data_loader):
        #print("Batch nro {}".format(i))

        mesh_frag = mesh_frag.float().to(available_device)
        mesh_complete = mesh_complete.float().to(available_device)
         
        output_g_encode = G_encode_decode.forward_encode(mesh_frag)
        Z = torch.Tensor(BSIZE, (Z_INTERN_SPACE - Z_LATENT_SPACE)).normal_(0, 0.33).to(available_device)
        output_g_encode_z = G_encode_decode.normalized_vector(output_g_encode, Z)
        fake = G_encode_decode.forward_decode(output_g_encode_z) 
        fake = (fake > 0).type(torch.int)
        fake = fake + mesh_frag.unsqueeze(1)
        
        for i, f in enumerate(fake.detach().cpu().numpy()):
            a = (f[0] == 1)
            #print(np.shape(a[a]))
            vox = Vox.from_dense(a)
            VoxWriter('fake_vox/v1_test_{}.vox'.format(i+1), vox).write()
        
        Z = torch.Tensor(BSIZE, (Z_INTERN_SPACE - Z_LATENT_SPACE)).normal_(0, 0.33).to(available_device)
        output_g_encode_z = G_encode_decode.normalized_vector(output_g_encode, Z)
        fake = G_encode_decode.forward_decode(output_g_encode_z) 
        fake = (fake > 0).type(torch.int)
        fake = fake + mesh_frag.unsqueeze(1)
        #fake = mesh_frag.unsqueeze(1)
        
        for i, f in enumerate(fake.detach().cpu().numpy()):
            a = (f[0] == 1)
            vox = Vox.from_dense(a)
            VoxWriter('fake_vox/v2_test_{}.vox'.format(i+1), vox).write()

        
        #fake[fake == 2] = 1
        break

In [13]:
crit_D = nn.CrossEntropyLoss()#nn.BCELoss()

In [21]:
y_real_ = torch.tensor(np.ones((BSIZE))).to(available_device).long()
y_fake_ = torch.tensor(np.zeros((BSIZE))).to(available_device).long()

In [23]:
D_fake = D_encode_decode(fake).view(BSIZE, 2)
D_fake_loss = crit_D(D_fake, y_fake_)

torch.Size([32, 512, 4, 4, 4])
torch.Size([32, 64])
torch.Size([32, 2])


In [ ]:
D_fake_loss

In [ ]:
512*4*4*4

In [ ]:
32*32